In [1]:
from azureml.core import Workspace
ws = Workspace.from_config() # 작업 공간의 셋팅 값을 가지고 올 수 있다
print('Workspace name:' + ws.name, 
    'Azure region:' + ws.location,
    'Subscription ID' + ws.subscription_id,
    'Resource Group:' + ws.resource_group
)

Workspace name:labuser2ml1 Azure region:eastus Subscription ID7ae06d59-97e1-4a36-bbfe-efb081b9b03b Resource Group:rg2


In [2]:
# 실험공간의 생성
from azureml.core import Experiment
experiment = Experiment(workspace = ws, name='diabets-experiment') # 워크스페이스를 지정을 해준다, experiment 변수에 저장을 시켜준다

In [3]:
from azureml.opendatasets import Diabetes # 당뇨병 데이터를 가지고 온다
from sklearn.model_selection import train_test_split # 정리하고 다듬을 꺼는 사이킷런을 가지고 온다, 그리고 데이터를 쪼개는 것을 가지고 온다

x_df = Diabetes.get_tabular_dataset().to_pandas_dataframe().dropna() # 애저에서 만든 pandas 데이터프레임을 가지고 온다, 빠져있는 데이터는 dropna 해준다
y_df = x_df.pop('Y') # 라벨 값을 가지고 오면서 x_df에는 Y를 없애준다(pop을 하였기 때문에 원본에는 Y의 값이 없는 것을 알 수 있다)

X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size = 0.2, random_state = 66) # 랜덤 시드를 정해준다
print(X_train)

     AGE  SEX   BMI     BP   S1     S2    S3    S4      S5   S6
440   36    1  30.0   95.0  201  125.2  42.0  4.79  5.1299   85
389   47    2  26.5   70.0  181  104.8  63.0  3.00  4.1897   70
5     23    1  22.6   89.0  139   64.8  61.0  2.00  4.1897   68
289   28    2  31.5   83.0  228  149.4  38.0  6.00  5.3132   83
101   53    2  22.2  113.0  197  115.2  67.0  3.00  4.3041  100
..   ...  ...   ...    ...  ...    ...   ...   ...     ...  ...
122   62    2  33.9  101.0  221  156.4  35.0  6.00  4.9972  103
51    65    2  27.9  103.0  159   96.8  42.0  4.00  4.6151   86
119   53    1  22.0   94.0  175   88.0  59.0  3.00  4.9416   98
316   53    2  27.7   95.0  190  101.8  41.0  5.00  5.4638  101
20    35    1  21.1   82.0  156   87.8  50.0  3.00  4.5109   95

[353 rows x 10 columns]


In [4]:
# 훈련을 하기 위해서 몇가지 셋팅을 해둔다
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error # MSE를 메트릭으로 사용한다
from sklearn.externals import joblib # 작업을 할 때마다 저장을 해준다.
import math

# alpha값을 어떻게 주느냐에 따라 결과가 달라진다
alphas = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0] # 0.1부터 1까지 alpha값을 리스트로 만들어 준다

for alpha in alphas:
    run = experiment.start_logging() # 실험이 얼마나 돌았는지 기록을 남겨야 한다 그리고 기록에서 어떤 옵션을 남겼는지 본다
    run.log('alpha_value', alpha) # 기록을 남기는 방법, 그럼 alpha값이 남겨진다

    model = Ridge(alpha=alpha) # alpha의 값이 0.1 부터 1.0까지 반복해서 돌아간다
    model.fit(X_train,y_train) # train의 feature값과 label값을 넣어준다

    y_pred = model.predict(X_test) # 테스트 데이터를 주고 예측을 시켜준다
    rmse = math.sqrt(mean_squared_error(y_test,y_pred)) #  MSE에 루트를 씌워서 RMSE로 만들어 준다 (한꺼번에 두개의 값을 보기 위해서) 실제 값과 예측값을 rmse로 비교해본다
    run.log('rmse',rmse) # rmse를 기록해둔다
     # 여기까지 했으면 실험과 결과가 끝나게 된다
    model_name = 'model_alpha_' + str(alpha) + '.pkl' # alpha값이 몇으로 만들어 졌는지 보는 것이다 그리고 pkl의 확장자를 붙여준다
    filename = 'outputs/' + model_name # 반복하면서 테스트하면서 로그를 찍고 파일도 저장이 되며 output의 폴더 밑에 쭉 저장이 된다
    
    joblib.dump(value=model,filename=filename) # dump를 하면 저장을 하게 된다
    run.upload_file(name=model_name, path_or_stream=filename) # 파일로도 저장이 되어있고 애저에도 저장이 된다
    run.complete() # 작업이 끝난거를 명시를 해준다
    
    print(f'{alpha} experiment completed')

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


0.1 experiment completed
0.2 experiment completed
0.3 experiment completed
0.4 experiment completed
0.5 experiment completed
0.6 experiment completed
0.7 experiment completed
0.8 experiment completed
0.9 experiment completed
1.0 experiment completed


In [5]:
experiment

Name,Workspace,Report Page,Docs Page
diabets-experiment,labuser2ml1,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
experiment.get_runs()

<generator object Run._rehydrate_runs at 0x7fdb34026d60>

In [7]:
# Best Model 탐색 후 다운로드

minimum_rmse_runid = None
minimum_rmse = None

for run in experiment.get_runs(): # 10번의 실행 결과를 뽑아낼 때 사용한다.
    run_matrics = run.get_metrics() # 실험의 0.1부터 1.0의 값이 들어가 있다.
    run_details = run.get_details()

    run_rmse = run_matrics['rmse'] # run_matrics에서 rmse를 가지고 오는 것이다.
    run_id = run_details['runId']

    if minimum_rmse is None: # 제일 처음 실행할 때, None부터 시작한다
        minimum_rmse = run_rmse # 실행한 run_rmse값을 넣어준다
        minimum_rmse_runid = run_id
    else:
        if run_rmse < minimum_rmse: # rmse가 minimum_rmse보다 작다면 값알 바꾸어준다
            minimum_rmse = run_rmse
            minimum_rmse_runid = run_id
print('Best run_id:' + minimum_rmse_runid)
print('Best rmse:' + str(minimum_rmse)) # 수치로 되어 있는 것을 문자로 바꾸어 준다.


# 김영욱 선생님이 작성해주신 코드
# # Best model 탐색 후 다운로드
# minium_rmse_runid = None
# minium_rmse = None

# for run in experiment.get_runs():
#     run_metrics = run.get_metrics()
#     run_details = run.get_details()
#     run_rmse = run_metrics['rmse']
#     run_id = run_details['runId']
#     if minium_rmse is None:
#         minium_rmse = run_rmse
#         minium_rmse_runid = run_id
#     else:
#         if run_rmse < minium_rmse:
#             minium_rmse = run_rmse
#             minium_rmse_runid = run_id
# print('Best run_id:' + minium_rmse_runid)
# print('Best rmse:' + str(minium_rmse))

Best run_id:1a3a0679-0625-4086-b047-e5c5b7f16f93
Best rmse:56.605203313391435


In [8]:
from azureml.core import Run # Run이라는 패키지를 불러온다
best_run = Run(experiment = experiment, run_id = minimum_rmse_runid) # 실험과 실행했던 아이디를 가지고 불러온다

best_run.download_file(name = str(best_run.get_file_names()[0])) # 파일이름은 문자열로 가지고 온다(첫번째 파일만 다운받아준다)

In [9]:
# 지금 현재 파일을 csv파일로 저장해준다
import numpy as np
from azureml.core import Dataset

np.savetxt('features.csv', X_train, delimiter = ',')
np.savetxt('labels.csv', y_train, delimiter = ',') # labels도 저장을 해둔다

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./features.csv', './labels.csv'],
                        target_path='diabetes-experiment/', # 타깃을 당뇨병으로 하였다
                        overwrite=True)
input_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'diabetes-experiment/features.csv')])
output_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'diabetes-experiment/labels.csv')])

# 이걸 실행하면 model_alpha_0.1.pkl에서 사용했던 features.csv와 labels.csv파일이 나오게 된다

######################################################################################################################
# import numpy as np
# from azureml.core import Dataset

# np.savetxt('features.csv', X_train, delimiter=',')
# np.savetxt('labels.csv', y_train, delimiter=',')

# datastore = ws.get_default_datastore()
# datastore.upload_files(files=['./features.csv', './labels.csv'],
#                        target_path='diabetes-experiment/',
#                        overwrite=True)

# input_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'diabetes-experiment/features.csv')])
# output_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'diabetes-experiment/labels.csv')])

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 2 files
Uploading ./features.csv
Uploaded ./features.csv, 1 files out of an estimated total of 2
Uploading ./labels.csv
Uploaded ./labels.csv, 2 files out of an estimated total of 2
Uploaded 2 files
Uploading an estimated of 2 files
Uploading ./features.csv
Uploaded ./features.csv, 1 files out of an estimated total of 2
Uploading ./labels.csv
Uploaded ./labels.csv, 2 files out of an estimated total of 2
Uploaded 2 files
Uploading an estimated of 2 files
Uploading ./features.csv
Uploaded ./features.csv, 1 files out of an estimated total of 2
Uploading ./labels.csv
Uploaded ./labels.csv, 2 files out of an estimated total of 2
Uploaded 2 files
Uploading an estimated of 2 files
Uploading ./features.csv
Uploaded ./features.csv, 1 files out of an estimated total of 2
Uploading ./labels.csv
Uploaded ./labels.csv, 2 files out of an estimated total of 2
Uploaded 2 files
Uploading an estimated of 2 files
Uploading ./features.csv
Uploaded ./features.csv, 1 files out of a

In [10]:
import sklearn

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = Model.register(workspace=ws,
                       model_name='diabetes-experiment-model',
                       model_path=f"./{str(best_run.get_file_names()[0])}",
                       model_framework=Model.Framework.SCIKITLEARN,  
                       model_framework_version=sklearn.__version__,  
                       sample_input_dataset=input_dataset,
                       sample_output_dataset=output_dataset,
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                       description='Ridge regression model to predict diabetes progression.',
                       tags={'area': 'diabetes', 'type': 'regression'})

print('Name:', model.name)
print('Version:', model.version)

Registering model diabetes-experiment-model
Name: diabetes-experiment-model
Version: 1


In [11]:
service_name = 'diabetes-service'

service = Model.deploy(ws, service_name, [model], overwrite = True) # deploy 명령을 사용해서 배포할 수 있다. 당뇨병 서비스로 디플로이 해본다 리스트 타입으로 모델을 배포한다
# 배포하기 위한 준비가 끝남
service.wait_for_deployment(show_output = True) # 서비스를 배포하는 것 까지 할 수 있다.

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-11-04 05:15:40+00:00 Creating Container Registry if not exists..
2022-11-04 05:25:41+00:00 Registering the environment.
2022-11-04 05:25:44+00:00 Uploading autogenerated assets for no-code-deployment..
2022-11-04 05:25:53+00:00 Building image..
2022-11-04 05:36:12+00:00 Generating deployment configuration.
2022-11-04 05:36:14+00:00 Submitting deployment to compute..
2022-11-04 05:36:22+00:00 Checking the status of deployment diabetes-service..
2022-11-04 05:38:20+00:00 Checking the status of inference endpoint diabetes-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [12]:
import json # json타입으로 주기 위해 json타입을 쓴다

input_payload = json.dumps({     
    'data': X_train[0:2].values.tolist(),     
    # 'data': X_test[0:2].values.tolist(),     # test에 대한 것을 예측할 수 있다.
    'method': 'predict'  # predict을 달라고 해준다
}) 

output = service.run(input_payload)  # 예측한 결과를 output으로 돌려준다

print(output) # 마자막에 여측한 결과를 출력해준다

{'predict': [204.94506937062147, 74.4641225933554]}
